In [42]:
import cv2
from PIL import Image
import io
import base64
from langchain.llms import Ollama
from langchain.tools import tool
from langchain_experimental.llms.ollama_functions import OllamaFunctions

@tool
def capture_and_analyze_image() -> str:
    """보다"""
    # 카메라 캡처
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    cap.release()

    if not ret:
        return "카메라에서 이미지를 캡처하는 데 실패했습니다."

    # 캡처한 이미지 저장
    cv2.imwrite('./captured/captured_image.jpg', frame)
    print("이미지가 저장되었습니다: captured_image.jpg")

    # OpenCV 이미지를 PIL 이미지로 변환
    img_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # 이미지를 base64로 인코딩
    buffered = io.BytesIO()
    img_pil.save(buffered, format="JPEG")
    img_str = base64.b64encode(buffered.getvalue()).decode()

    # llava 모델 초기화 및 이미지 분석
    llava = Ollama(model="llava-phi3")
    response = llava("Analyze this image and describe what you see.", images=[img_str])

    return response

# LangChain LLM 초기화 및 도구 바인딩
llm = OllamaFunctions(model="Bllossom:q8_0", format="json")
llm_with_tools = llm.bind_tools([capture_and_analyze_image])

# 함수 사용 예시
# response = llm_with_tools.invoke("카메라로 사진을 찍고 분석해주세요.")
# print(response)

In [43]:
capture_and_analyze_image.description

'보다'

In [52]:
from langchain_core.messages import HumanMessage, ToolMessage
query = "도구를 활용하여 뭐가 보이는지 알려줘"
messages = [HumanMessage(query)]
ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)
for tool_call in ai_msg.tool_calls:
    selected_tool = {"seeing image": capture_and_analyze_image}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='도구를 활용하여 뭐가 보이는지 알려줘'),
 AIMessage(content='이미지를 분석할 수 있는 도구는 제공되지 않습니다. 다른 질문이나 도움이 필요하면 말씀해 주세요.', id='run-3db6be62-d51c-4982-b919-c407a9d19c39-0')]

In [32]:
print(messages[2].content)

IndexError: list index out of range

In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from transformers import pipeline, AutoConfig
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage
from ollama_function import get_capture
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_core.tools import tool

/Users/leehwan/miniconda3/envs/ml/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# LangChain이 지원하는 다른 채팅 모델을 사용합니다. 여기서는 Ollama를 사용합니다.
llm = OllamaFunctions(model="Bllossom:q8_0")

model_path = "../../../BART/final_model"
model_name = "gogamza/kobart-base-v2"
config = AutoConfig.from_pretrained(model_name, num_labels=2)
nlg_pipeline = pipeline('text2text-generation', model=model_path, tokenizer=model_name, config=config, device='mps')

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI Assistant. Your name is 'ENFP'. You must answer in Korean.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

def generate_text(text):
    text = f"enfp 말투로 변환:{text}"
    out = nlg_pipeline(text, max_length=60)
    return out[0]['generated_text']

def format_output(text):
    return text.strip()

@tool
def get_img():
    """눈 앞에 보이는 것을 분석해줍니다."""
    query = "눈 앞에 보이는 것을 카메라로 찍어 설명해주세요"
    return get_capture(query=query)


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
llm_with_tools = llm.bind_tools(tools=[get_img])
query = "눈 앞에 보이는 것을 카메라로 찍어 설명해주세요"
response = llm_with_tools.invoke(query)
print(response)

In [ ]:
chain = prompt | llm | StrOutputParser() | generate_text | format_output | StrOutputParser()

In [53]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful AI Vision Assistant.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
)

In [89]:
def re(prompt):
    prompt1 = prompt.to_messages
    return {"english_word": prompt1}

In [90]:
chain = prompt | re

In [91]:
print(chain.invoke(["test"]))

{'english_word': <bound method ChatPromptValue.to_messages of ChatPromptValue(messages=[SystemMessage(content='You are a helpful AI Vision Assistant.'), HumanMessage(content='test')])>}


In [78]:
prompt

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful AI Vision Assistant.')), MessagesPlaceholder(variable_name='messages')])